# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcdf8921370>
├── 'a' --> tensor([[-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661]])
└── 'x' --> <FastTreeValue 0x7fcdf89216a0>
    └── 'c' --> tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                        [ 0.2865,  0.0586, -0.6048, -1.0196],
                        [-1.8792,  0.2377,  0.2134,  0.1113]])

In [4]:
t.a

tensor([[-0.9872,  0.3353,  1.0311],
        [ 0.2495, -0.1852,  0.7661]])

In [5]:
%timeit t.a

77.8 ns ± 0.258 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcdf8921370>
├── 'a' --> tensor([[ 0.9678, -1.4829, -0.0901],
│                   [ 1.1839, -0.2621, -0.4483]])
└── 'x' --> <FastTreeValue 0x7fcdf89216a0>
    └── 'c' --> tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                        [ 0.2865,  0.0586, -0.6048, -1.0196],
                        [-1.8792,  0.2377,  0.2134,  0.1113]])

In [7]:
%timeit t.a = new_value

84.4 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661]]),
    x: Batch(
           c: tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                      [ 0.2865,  0.0586, -0.6048, -1.0196],
                      [-1.8792,  0.2377,  0.2134,  0.1113]]),
       ),
)

In [10]:
b.a

tensor([[-0.9872,  0.3353,  1.0311],
        [ 0.2495, -0.1852,  0.7661]])

In [11]:
%timeit b.a

68.8 ns ± 0.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2561, -0.7506, -1.1251],
               [ 1.6792,  0.9943, -0.6769]]),
    x: Batch(
           c: tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                      [ 0.2865,  0.0586, -0.6048, -1.0196],
                      [-1.8792,  0.2377,  0.2134,  0.1113]]),
       ),
)

In [13]:
%timeit b.a = new_value

581 ns ± 1.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

970 ns ± 2.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 9.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 468 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

169 µs ± 8.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcdf8914220>
├── 'a' --> tensor([[[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]],
│           
│                   [[-0.9872,  0.3353,  1.0311],
│                    [ 0.2495, -0.1852,  0.7661]]])
└── 'x' --> <FastTreeValue 0x7fcdf8921d60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.4 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcd41bb7af0>
├── 'a' --> tensor([[-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661],
│                   [-0.9872,  0.3353,  1.0311],
│                   [ 0.2495, -0.1852,  0.7661]])
└── 'x' --> <FastTreeValue 0x7fcd41bb7a30>
    └── 'c' --> tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                        [ 0.2865,  0.0586, -0.6048, -1.0196],
                 

In [23]:
%timeit t_cat(trees)

37 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.2 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]],
       
               [[-0.9872,  0.3353,  1.0311],
                [ 0.2495, -0.1852,  0.7661]]]),
    x: Batch(
           c: tensor([[[ 0.1433,  0.1027,  2.0063,  1.3426],
                       [ 0.2865,  0.0586, -0.6048, -1.0196],
                       [-1.8792,  0.2377,  0.2134,  0.1113]],
         

In [26]:
%timeit Batch.stack(batches)

91.4 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661],
               [-0.9872,  0.3353,  1.0311],
               [ 0.2495, -0.1852,  0.7661]]),
    x: Batch(
           c: tensor([[ 0.1433,  0.1027,  2.0063,  1.3426],
                      [ 0.2865,  0.0586, -0.6048, -1.0196],
                      [-1.8792,  0.2377,  0.2134,  0.1113],
                      [ 0.1433,  0.1027,  2.0063,  1.3426],
                      [ 0.2865,  

In [28]:
%timeit Batch.cat(batches)

167 µs ± 335 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

388 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
